### Initialize

In [1]:
import requests
import pandas as pd
import os
import sys
from datetime import datetime

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))

from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path
import src.esh_client as esh

base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]


def execute_search_query(query):
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[
                ['firstName', 'lastName', 'age']]
            df.rename(columns={'firstName': 'First Name',
                      'lastName': 'Last Name', 'age': 'Age'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())



### Fuzzy-search for first name "Alwx"
-> It is searched for the first name with 70 percent match

In [2]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Alwx',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))


Total count: 1


,First Name,Last Name,Age
0,Alex,Kennedy,57


## Fuzzy-search with first name and lastname 
-> It is searchef for the first name and last name with a 70 percent match

In [3]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Alwx',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            ),
            esh.Comparison(
                property=esh.Property(property=['lastName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Konnedy',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))

Total count: 1


,First Name,Last Name,Age
0,Alex,Kennedy,57


## Fuzzy-search with place of birth and first name
-> It is searched for a birst with place of birth and first name with a 70 percent match

In [4]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['placeOfBirth']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Sutggart',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            ),
            esh.Comparison(
                property=esh.Property(property=['firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Hildo',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))


Total count: 1


,First Name,Last Name,Age
0,Hilda,Friedel,51


## Fuzzy-search with nationality, place of birth first name and last name
--> This fuzzy search includes multiple conditions with a 70 percent match

In [5]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Ivon',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            ),
            esh.Comparison(
                property=esh.Property(property=['lastName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Kowalsko',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            ),
            esh.Comparison(
                property=esh.Property(property=['placeOfBirth']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Warsow',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            ),
            esh.Comparison(
                property=esh.Property(property=['nationality']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Uk',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))


Total count: 1


,First Name,Last Name,Age
0,Ivan,Kowalski,65


## Search for persons who are approx. 30 years old
--> It is searched for persons who are approx. 30 years old. Therefore the field "age" is set as property and the term "30" is entered. Due to the age 30 has just to digits the fuzzinesThreshold is set to 0.5 precision (50% match).

In [6]:
so = esh.EshObject(
    count=True,
    top=2,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['age']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value=30,
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.5)
                )
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))


Total count: 2


,First Name,Last Name,Age
0,Joseph,Kennedy,36
1,Jerry,Roberts,37
